In [ ]:
!pip install deap update_checker tqdm stopit tpot

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 5.8 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=e5cc0b93f60e39bb0d871d4321e1a38ebb27087f1dcf5c1227c312b1a0cd5e06
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.

In [ ]:
import pandas as pd
import numpy as np
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder


df=pd.read_csv("adult.csv")

X = df.drop('income', axis=1)
y = df['income']

###TPOT needs only numeric columns###

# One-Hot Encode categorical variables
categorical_cols = X.select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_cols)],
    remainder='passthrough')

X = preprocessor.fit_transform(X)

#label encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)



X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=None)


In [ ]:
# initializing tpot with parameters
#max_time_mins=None (per default) - here max time 4h
#config_dic due to sparse matrix

tpot = TPOTClassifier(generations=5, population_size=50,
                     cv=5, config_dict='TPOT sparse', verbosity=2, n_jobs=-1, max_time_mins=240,
                     periodic_checkpoint_folder='/content/results')

In [ ]:
#starting the training
tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/50 [00:00<?, ?pipeline/s]


6.71 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.5, max_depth=2, min_child_weight=14, n_estimators=100, n_jobs=1, subsample=1.0, verbosity=0)


TPOTClassifier(config_dict='TPOT sparse', generations=5, max_time_mins=2,
               n_jobs=-1, periodic_checkpoint_folder='/content/results',
               population_size=50, verbosity=2)

In [ ]:
import sklearn.metrics as skm
from sklearn.metrics import roc_auc_score

#prediction
y_pred = tpot.predict(X_test)

# prediction for the AUC-ROC-Score by only using the positive classes
y_pred_proba = tpot.predict_proba(X_test)[:, 1]


print("RESULTS OF BEST MODEL:\n")

print(f"F1-Score:                   {skm.f1_score(y_test, y_pred)}")
print(f"AUC-ROC Score:              {roc_auc_score(y_test, y_pred_proba)}")
print(f"Accuracy:                   {skm.accuracy_score(y_test, y_pred)}")
print(f"Precision:                  {skm.precision_score(y_test, y_pred)}")
print(f"Recall:                     {skm.recall_score(y_test, y_pred)}")

RESULTS OF BEST MODEL:

F1-Score:                   0.6944127708095781
AUC-ROC Score:              0.9207286405693997
Accuracy:                   0.8683208451050239
Precision:                  0.7689393939393939
Recall:                     0.6330561330561331


In [ ]:
from google.colab import files
# output of values to dictionary
metrics_dict = {
    'Metric': [
        'F1-Score',
        'AUC-ROC Score',
        'Accuracy',
        'Precision',
        'Recall',
    ],
    'Value': [
        skm.f1_score(y_test, y_pred),
        roc_auc_score(y_test, y_pred_proba),
        skm.accuracy_score(y_test, y_pred),
        skm.precision_score(y_test, y_pred),
        skm.recall_score(y_test, y_pred)
    ]
}

#to df
metrics_df = pd.DataFrame(metrics_dict)


print(metrics_df)

metrics_df.to_csv('tpot_classification_preprosessing_metrics.csv', index=False)
files.download('tpot_classification_preprosessing_metrics.csv')

In [ ]:
tpot.export('tpot_classification_normal_model')